## Pytorch Lightning CIFAR10

In [1]:
from typing import Any

# Basic torch import
import torch
import torch.nn as nn

# torch Lightning import
import lightning.pytorch as pl
import torchmetrics

# Data Management
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [5]:
# 코드작성
class LCIFAR10Model(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # Metric
        self.train_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
        self.val_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)
        self.test_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)

        # Feature Extractor
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32 ,out_channels=32, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64 ,kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)

        # Classification
        self.linear1 = nn.Linear(128*4*4, 256)
        self.linear2 = nn.Linear(256, 10)

    def forward(self, x):

        # Feature Extractor
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.maxpool1(x)

        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = self.maxpool2(x)

        x = torch.relu(self.conv5(x))
        x = torch.relu(self.conv6(x))
        x = self.maxpool3(x)

        # Flatten
        x = nn.Flatten()(x)

        # Classification
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)

        # Softmax Distribution
        x = torch.log_softmax(x, dim=1)
        return x

    def cross_entropy(self, logits, labels):
        # Negative Log Likelihood Loss

        # Cross Entropy Loss = log_softmax + NLL Loss
        return nn.functional.nll_loss(logits, labels)

    # 학습루프
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy(logits, y)
        acc = self.train_acc(logits, y)
        self.log('train_loss', loss)
        self.train_acc(logits, y)
        self.log('train_acc', acc, prog_bar=True, on_step=True, on_epoch=True)

        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy(logits, y)
        acc = self.val_acc(logits, y)
        self.log('val_loss', loss)
        self.val_acc(logits, y)
        self.log('val_acc',acc, prog_bar=True, on_step=True, on_epoch=True)

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.cross_entropy(logits, y)
        acc = self.test_acc(logits, y)
        self.log('test_loss', loss)
        self.test_acc(logits, y)
        self.log('test_acc', acc , prog_bar=True, on_step=True, on_epoch=True)

    # optim
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr= 0.001)
        return optimizer

## 데이터 가공

In [3]:
# 데이터 정규화
transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                      (0.247, 0.243, 0.261))])

# 데이터 준비
train_data = CIFAR10(root="/Dataset", train=True, download=True, transform=transforms)
train_data, val_data = random_split(train_data, [0.9, 0.1])

test_data = CIFAR10(root="/Dataset", train=False, download=True, transform=transforms)

# 데이터로더
train_dataloader = DataLoader(train_data, batch_size= 128, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size= 128)
test_dataloader = DataLoader(test_data, batch_size= 128)

100%|██████████| 170M/170M [00:47<00:00, 3.56MB/s] 


Extracting /Dataset\cifar-10-python.tar.gz to /Dataset
Files already downloaded and verified


In [6]:
model = LCIFAR10Model()
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=50)

trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type               | Params | Mode 
----------------------------------------------------------
0  | train_acc | MulticlassAccuracy | 0      | train
1  | val_acc   | MulticlassAccuracy | 0      | train
2  | test_acc  | MulticlassAccuracy | 0      | train
3  | conv1     | Conv2d             | 896    | train
4  | conv2     | Conv2d             | 9.2 K  | train
5  | maxpool1  | MaxPool2d          | 0      | train
6  | conv3     | Conv2d             | 18.5 K | train
7  | conv4     | Conv2d             | 36.9 K | train
8  | maxpool2  | MaxPool2d          | 0      | train
9  | conv5     | Conv2d             | 73.9 K | train
10 | conv6     | Conv2d             | 147 K  | train
11 | maxpool3  | MaxPool2d          | 0      | train
12 | linear1   | Linear             | 524 K  | train
13 | linear2   | Linear             | 2.

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\user\anaconda3\envs\namduhus_GPU\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


In [7]:
trainer.test(model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\user\anaconda3\envs\namduhus_GPU\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.7591000199317932     │
│         test_loss         │    1.9095467329025269     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.9095467329025269, 'test_acc_epoch': 0.7591000199317932}]